In [1]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import warnings

### Load the data

In [ ]:
output_file=r"../data/synthetic_data_baseline_prompt.csv.csv"

def load_data(output_file, latency_file=None):
    """Load the CSV files."""
    df_output = pd.read_csv(output_file)
    
    if latency_file:
        df_latency = pd.read_csv(latency_file)
    else:
        df_latency = None
    
    print("=== DATA OVERVIEW ===")
    print(f"Total records: {len(df_output)}")
    print(f"Total users: {df_output['user_id'].nunique()}")
    print(f"Actions: {sorted(df_output['rl_action'].unique())}\n")
    
    return df_output, df_latency

# %%
def parse_ratings(rating_str):
    """Parse rating string to list."""
    try:
        ratings = ast.literal_eval(rating_str)
        return ratings if ratings else []
    except:
        return []

def prepare_rating_data_for_boxplot(df):
    """Prepare rating data for boxplot visualization."""
    rating_data = []
    
    for _, row in df.iterrows():
        ratings = parse_ratings(row['actual_rating'])
        action = row['rl_action']
        
        for rating in ratings:
            rating_data.append({
                'action': action,
                'rating': rating
            })
    
    return pd.DataFrame(rating_data)